# 01 — Intent Classification (rules)
**Date:** 2025-08-09

Keep it deterministic and testable. Adjust keywords as your domain grows.


In [1]:
#r "nuget: Microsoft.Recognizers.Text, 1.8.13"
#r "nuget: Microsoft.Recognizers.Text.DateTime, 1.8.13"
#r "nuget: FuzzySharp, 2.0.2"
#r "nuget: RestSharp, 112.1.0"
#r "nuget: Microsoft.Extensions.Caching.Memory, 9.0.8"
#r "nuget: System.Text.Json, 9.0.0"


Installed Packages FuzzySharp, 2.0.2 Microsoft.Extensions.Caching.Memory, 9.0.8 Microsoft.Recognizers.Text, 1.8.13 Microsoft.Recognizers.Text.DateTime, 1.8.13 RestSharp, 112.1.0 System.Text.Json, 9.0.0

In [ ]:
using System;
using System.Collections.Generic;
using System.Linq;
using System.Text.Json;
using Microsoft.Recognizers.Text;
using Microsoft.Recognizers.Text.DateTime;
using FuzzySharp;
using RestSharp;

public enum Intent { GetContactInfo, FilterByHireDate, FilterByRole, Unknown }

public record Slots(
    string[]? Names = null,
    DateTime? Date = null,
    (DateTime Start, DateTime End)? Range = null,
    string? Operator = null,
    string? Department = null,
    string? Role = null
);

public record QuerySpec(Intent Intent, Slots Slots);

public record Employee(
    string DisplayName,
    string Email,
    string Department,
    string Role,
    DateTime OriginalHireDate
);

var employees = new List<Employee> {
    new("Rick Sanchez",   "rick.sanchez@company.com",   "Engineering", "Staff Engineer",  new DateTime(2015,  5, 10)),
    new("Morty Smith",    "morty.smith@company.com",    "Engineering", "Engineer I",      new DateTime(2023, 10, 12)),
    new("Summer Smith",   "summer.smith@company.com",   "Product",     "PM",              new DateTime(2021,  2,  1)),
    new("Beth Smith",     "beth.smith@company.com",     "HR",          "HR Manager",      new DateTime(2019,  7,  3)),
    new("Jerry Smith",    "jerry.smith@company.com",    "Sales",       "Account Manager", new DateTime(2022,  9, 15))
};

Console.WriteLine($"Loaded demo employees: {employees.Count}");


In [ ]:
static Intent ClassifyIntent(string query)
{
    var q = query.ToLowerInvariant();
    if (q.Contains("email") || q.Contains("contact")) return Intent.GetContactInfo;
    if (q.Contains("hire") && (q.Contains("before") || q.Contains("after") || q.Contains("between"))) return Intent.FilterByHireDate;
    if (q.Contains("manager") || q.Contains("engineer") || q.Contains("director") || q.Contains("role")) return Intent.FilterByRole;
    return Intent.Unknown;
}

Console.WriteLine(ClassifyIntent("what are rick and summer's emails?"));
Console.WriteLine(ClassifyIntent("show managers hired after 2020"));
Console.WriteLine(ClassifyIntent("who are the engineers?"));
